# 路由链
- 路由链支持创建一个非确定性链， 由LLM来选择下一步
- 链内的多个prompts模板描述了不同的提示请求

## RouterChain

In [15]:
# RouterChain 
from langchain.prompts import PromptTemplate

# 物理链
physisc_template = """你是一位非常聪明的物理教授。\n
你擅长以简洁易懂的方式回答物理问题。\n
当您不知道问题答案的时候，你会坦率承认不知道。\n
下面是一个问题：
{input}
"""
physisc_prompt = PromptTemplate.from_template(physisc_template)

#数学链
math_template = """你是一位非常优秀明的数学教授。\n
您擅长回答数学问题。\n
您之所以如此优秀，是因为您能够将困难问题分解成组成的部分，回答这些部分，然后将他们组合起来，回答更广泛的问题。\n
下面是一个问题:
{input}
"""

match_prompt = PromptTemplate.from_template(math_template)

from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.chains  import LLMChain

prompt_infos = [
    {
        "name": "physisc",
        "description": "擅长回答物理问题",
        "prompt_template": physisc_template,
    },
    {
        "name": "math",
        "description": "擅长回答数学问题",
        "prompt_template": math_template,
    }
]

llm = ChatOpenAI(temperature=0)

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]

    prompt_template = p_info["prompt_template"]

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=['input']
    )

    chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

    destination_chains[name] = ConversationChain(llm=llm, output_key="text", verbose=True)


#router chain

from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router import MultiPromptChain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
desctination_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=desctination_str)
# print(MULTI_PROMPT_ROUTER_TEMPLATE)
# print(router_template)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)


default_chain = ConversationChain(
    llm = llm,
    output_key="text"
)

#多链接连接
chain = MultiPromptChain(
    router_chain = router_chain,
    destination_chains = destination_chains,
    default_chain = default_chain,
    verbose=True
)

#run
chain.run("什么是牛顿定律？中文输出")



> Entering new MultiPromptChain chain...
physisc: {'input': '什么是牛顿定律？中文输出'}

> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 什么是牛顿定律？中文输出
AI:

> Finished chain.

> Finished chain.


' 牛顿定律是经典力学的基础，由英国物理学家艾萨克·牛顿在17世纪提出的三个基本定律。第一定律是惯性定律，描述了物体在没有外力作用下保持静止或匀速直线运动的状态。第二定律是运动定律，描述了物体受力时加速度与作用力成正比的关系。第三定律是作用与反作用定律，描述了两个物体之间相互作用时，彼此施加的力大小相等、方向相反。'

In [16]:
chain.run("2x2=?")



> Entering new MultiPromptChain chain...


math: {'input': '2x2=?'}

> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 2x2=?
AI:

> Finished chain.

> Finished chain.


'4'

In [17]:
chain.run("床前明月光，下一句")



> Entering new MultiPromptChain chain...


None: {'input': '床前明月光，下一句'}
> Finished chain.


'疑是地上霜。这是一首古诗《静夜思》的开头，作者是唐代诗人李白。这首诗描述了一个人在夜晚思考人生的哲理。'

###  TransFormation 文档转换链

In [35]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """对下面的文字进行总结：
    {output_text}
    总结:
    """
)

with open("letter.txt") as f:
    letters = f.read()


from langchain.chains import(
    LLMChain,
    SimpleSequentialChain,
    TransformChain
)
from langchain.chat_models import ChatOpenAI


def transform_func(inputs:dict) -> dict:
    text = inputs["text"]
    shortened_text = "\n\n".join(text.split("\n\n")[:3])
    return {"output_text": shortened_text}

#文档转换链
tranform_chain = TransformChain(
    input_variables=["text"],
    output_variables=["output_text"],
    transform=transform_func
)

template = """对下面的文字进行总结:
{output_text}


总结:"""

prompt = PromptTemplate(
    input_variables = ["output_text"],
    template=template
)

print(prompt)

llm_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=prompt
)

#使用顺序链链接起来
squential_chian = SimpleSequentialChain(
    chains=[tranform_chain,llm_chain],
    verbose=True
)

squential_chian.run(letters)

input_variables=['output_text'] template='对下面的文字进行总结:\n{output_text}\n\n\n总结:'


> Entering new SimpleSequentialChain chain...
[Generated with ChatGPT]

Confidential Document - For Internal Use Only

Date: July 1, 2023
这是一份机密文件，仅供内部使用，日期为2023年7月1日。

> Finished chain.


'这是一份机密文件，仅供内部使用，日期为2023年7月1日。'